## Generative AI Use Case: Prompt Engineering


# Table of Contents

- [ 1 - Set up required Dependencies](#1)
- [ 2 - Data and Model Setup](#2)
- [ 3 - Zero Shot Inference](#3)
- [ 4 - One Shot Inference](#4)
- [ 5 - Few Shot Inference](#5)
- [ 6 - Tuning Few Shot Inference using Generative Configuration](#6)
- [ 7 - Medical Informational ChatBot](#7)
- [ 8 - Self Conversation Chat Information Retrieval](#8)


<a name='1'></a>
### 1 - Set up required Dependencies

In [1]:
%pip install --upgrade pip
%pip install \
    transformers==4.27.2 \
    datasets==2.11.0  --quiet
!pip install ipywidgets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 12.9 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 67.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 13.4 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM
from transformers import AutoTokenizer
from transformers import GenerationConfig
import json
import ipywidgets as widgets
from IPython.display import display, clear_output

<a name='2'></a>
### 2 - Data and Model Setup

In [3]:
huggingface_dataset_name = "knkarthick/dialogsum"

dataset = load_dataset(huggingface_dataset_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/knkarthick___csv/knkarthick--dialogsum-cd36827d3490488d/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 12460
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 1500
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 500
    })
})

In [5]:
print("dataset['test'][0]")

dataset['test'][0]


In [6]:
example_indices = [1, 4]

dash_line = '-'.join('' for x in range(100))

for i, index in enumerate(example_indices):
    print(dash_line)
    print('Example ', i + 1)
    print(dash_line)
    print('INPUT DIALOGUE:')
    print(dataset['test'][index]['dialogue'])
    print(dash_line)
    print('BASELINE HUMAN SUMMARY:')
    print(dataset['test'][index]['summary'])
    print(dash_line)
    print()

---------------------------------------------------------------------------------------------------
Example  1
---------------------------------------------------------------------------------------------------
INPUT DIALOGUE:
#Person1#: Ms. Dawson, I need you to take a dictation for me.
#Person2#: Yes, sir...
#Person1#: This should go out as an intra-office memorandum to all employees by this afternoon. Are you ready?
#Person2#: Yes, sir. Go ahead.
#Person1#: Attention all staff... Effective immediately, all office communications are restricted to email correspondence and official memos. The use of Instant Message programs by employees during working hours is strictly prohibited.
#Person2#: Sir, does this apply to intra-office communications only? Or will it also restrict external communications?
#Person1#: It should apply to all communications, not only in this office between employees, but also any outside communications.
#Person2#: But sir, many employees use Instant Messaging to c

In [7]:
model_name='google/flan-t5-base'
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [9]:
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

<a name='3'></a>
### 3 - Zero Shot Inference

In [10]:
for i, index in enumerate(example_indices):
    dialogue = dataset['test'][index]['dialogue']
    summary = dataset['test'][index]['summary']

    prompt = f"""
Dialogue:

{dialogue}

What was going on?
"""

    inputs = tokenizer(prompt, return_tensors='pt')
    output = tokenizer.decode(
        model.generate(
            inputs["input_ids"],
            max_new_tokens=50,
        )[0],
        skip_special_tokens=True
    )

    print(dash_line)
    print('Example ', i + 1)
    print(dash_line)
    print(f'INPUT PROMPT:\n{prompt}')
    print(dash_line)
    print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')
    print(dash_line)
    print(f'MODEL GENERATION - ZERO SHOT:\n{output}\n')

---------------------------------------------------------------------------------------------------
Example  1
---------------------------------------------------------------------------------------------------
INPUT PROMPT:

Dialogue:

#Person1#: Ms. Dawson, I need you to take a dictation for me.
#Person2#: Yes, sir...
#Person1#: This should go out as an intra-office memorandum to all employees by this afternoon. Are you ready?
#Person2#: Yes, sir. Go ahead.
#Person1#: Attention all staff... Effective immediately, all office communications are restricted to email correspondence and official memos. The use of Instant Message programs by employees during working hours is strictly prohibited.
#Person2#: Sir, does this apply to intra-office communications only? Or will it also restrict external communications?
#Person1#: It should apply to all communications, not only in this office between employees, but also any outside communications.
#Person2#: But sir, many employees use Instant Mess

<a name='4'></a>
### 4 - One Shot Inference

In [11]:
def make_prompt(example_indices_full, example_index_to_summarize):
    prompt = ''
    for index in example_indices_full:
        dialogue = dataset['test'][index]['dialogue']
        summary = dataset['test'][index]['summary']

        # The stop sequence '{summary}\n\n\n' is important for FLAN-T5. Other models may have their own preferred stop sequence.
        prompt += f"""
Dialogue:

{dialogue}

What was going on?
{summary}


"""

    dialogue = dataset['test'][example_index_to_summarize]['dialogue']

    prompt += f"""
Dialogue:

{dialogue}

What was going on?
{summary}
"""

    return prompt

In [12]:
example_indices_full = [40]
example_index_to_summarize = 1

one_shot_prompt = make_prompt(example_indices_full, example_index_to_summarize)

print(one_shot_prompt)


Dialogue:

#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.

What was going on?
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.



Dialogue:

#Person1#: Ms. Dawson, I need you to take a dictation for me.
#Person2#: Yes, sir...
#Person1#: This should go out as an intra-office memorandum to all employees by this afternoon. Are you ready?
#Person2#: Yes, sir. Go ahead.
#Person1#: Attention all staff... Effective immediately, all office communications are restricted to email correspondence and official memos. The use of Instant Message programs by employees during working hours is strictly prohibited.
#Person2#: Sir, does this apply to intra-offic

In [13]:
summary = dataset['test'][example_index_to_summarize]['summary']

inputs = tokenizer(one_shot_prompt, return_tensors='pt')
output = tokenizer.decode(
    model.generate(
        inputs["input_ids"],
        max_new_tokens=50,
    )[0],
    skip_special_tokens=True
)

print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')
print(dash_line)
print(f'MODEL GENERATION - ONE SHOT:\n{output}')

Token indices sequence length is longer than the specified maximum sequence length for this model (552 > 512). Running this sequence through the model will result in indexing errors


---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
In order to prevent employees from wasting time on Instant Message programs, #Person1# decides to terminate the use of those programs and asks Ms. Dawson to send out a memo to all employees by the afternoon.

---------------------------------------------------------------------------------------------------
MODEL GENERATION - ONE SHOT:
Tom will take a dictation for Person1#.


<a name='5'></a>
### 5 - Few Shot Inference

In [14]:
example_indices_full = [20, 80, 120, 50]
example_index_to_summarize = 100

few_shot_prompt = make_prompt(example_indices_full, example_index_to_summarize)

print(few_shot_prompt)


Dialogue:

#Person1#: What's wrong with you? Why are you scratching so much?
#Person2#: I feel itchy! I can't stand it anymore! I think I may be coming down with something. I feel lightheaded and weak.
#Person1#: Let me have a look. Whoa! Get away from me!
#Person2#: What's wrong?
#Person1#: I think you have chicken pox! You are contagious! Get away! Don't breathe on me!
#Person2#: Maybe it's just a rash or an allergy! We can't be sure until I see a doctor.
#Person1#: Well in the meantime you are a biohazard! I didn't get it when I was a kid and I've heard that you can even die if you get it as an adult!
#Person2#: Are you serious? You always blow things out of proportion. In any case, I think I'll go take an oatmeal bath.

What was going on?
#Person1# thinks #Person2# has chicken pox and warns #Person2# about the possible hazards but #Person2# thinks it will be fine.



Dialogue:

#Person1#: May, do you mind helping me prepare for the picnic?
#Person2#: Sure. Have you checked the wea

In [15]:
summary = dataset['test'][example_index_to_summarize]['summary']

inputs = tokenizer(few_shot_prompt, return_tensors='pt')
output = tokenizer.decode(
    model.generate(
        inputs["input_ids"],
        max_new_tokens=512,
    )[0],
    skip_special_tokens=True
)

print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')
print(dash_line)
print(f'MODEL GENERATION - FEW SHOT:\n{output}')

---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# and Mike have a disagreement on how to act out a scene. #Person1# proposes that Mike can try to act in #Person1#'s way.

---------------------------------------------------------------------------------------------------
MODEL GENERATION - FEW SHOT:
Person1 is about to make a prank.


<a name='6'></a>
### 6 - Tuning Few Shot Inference using Generative Configuration

In [20]:
# generation_config = GenerationConfig(max_new_tokens=50)
# generation_config = GenerationConfig(max_new_tokens=50, do_sample=True, temperature=0.5)
generation_config = GenerationConfig(max_new_tokens=50, do_sample=True, temperature=1.0)

inputs = tokenizer(few_shot_prompt, return_tensors='pt')
output = tokenizer.decode(
    model.generate(
        inputs["input_ids"],
        generation_config=generation_config,
    )[0],
    skip_special_tokens=True
)

print(dash_line)
print(f'MODEL GENERATION - FEW SHOT:\n{output}')
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')

---------------------------------------------------------------------------------------------------
MODEL GENERATION - FEW SHOT:
They want to make a prank on a married couple.
---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# and Mike have a disagreement on how to act out a scene. #Person1# proposes that Mike can try to act in #Person1#'s way.



<a name='7'></a>
### 7 - Medical Informational ChatBot

In [17]:
dataset2 = load_dataset("ShenRuililin/MedicalQnA")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/ShenRuililin___csv/ShenRuililin--MedicalQnA-d458888f37ba02ad/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [18]:
dataset2

DatasetDict({
    train: Dataset({
        features: ['input', 'output'],
        num_rows: 16412
    })
})

In [21]:
indices = [0, 3, 15, 200, 9722]

for i, index in enumerate(indices):
    print('Example ', i + 1)
    print('INPUT :',dataset2['train'][index]['input'])
    print('OUTPUT :',dataset2['train'][index]['output'])
    print(' ')

Example  1
INPUT : What is (are) Glaucoma ?
OUTPUT : Glaucoma is a group of diseases that can damage the eye's optic nerve and result in vision loss and blindness. While glaucoma can strike anyone, the risk is much greater for people over 60. How Glaucoma Develops  There are several different types of glaucoma. Most of these involve the drainage system within the eye. At the front of the eye there is a small space called the anterior chamber. A clear fluid flows through this chamber and bathes and nourishes the nearby tissues. (Watch the video to learn more about glaucoma. To enlarge the video, click the brackets in the lower right-hand corner. To reduce the video, press the Escape (Esc) button on your keyboard.) In glaucoma, for still unknown reasons, the fluid drains too slowly out of the eye. As the fluid builds up, the pressure inside the eye rises. Unless this pressure is controlled, it may cause damage to the optic nerve and other parts of the eye and result in loss of vision. Op

In [22]:
dash_line = '-'.join('' for x in range(100))

for i,index in enumerate(indices):
    input = dataset2['train'][index]

    prompt = f""" {input}
What is going on?
    """

    inputs = tokenizer(prompt, return_tensors='pt')
    zero_shot_output = tokenizer.decode(
        model.generate(
            inputs["input_ids"],
            max_new_tokens=50,
        )[0],
        skip_special_tokens=True
    )

    print('\nExample ', i + 1)
    print(f'\nINPUT PROMPT: {prompt}')
    print(f'MODEL - ZERO SHOT: {zero_shot_output}\n')
    print(dash_line)


Example  1

INPUT PROMPT:  {'input': 'What is (are) Glaucoma ?', 'output': "Glaucoma is a group of diseases that can damage the eye's optic nerve and result in vision loss and blindness. While glaucoma can strike anyone, the risk is much greater for people over 60. How Glaucoma Develops  There are several different types of glaucoma. Most of these involve the drainage system within the eye. At the front of the eye there is a small space called the anterior chamber. A clear fluid flows through this chamber and bathes and nourishes the nearby tissues. (Watch the video to learn more about glaucoma. To enlarge the video, click the brackets in the lower right-hand corner. To reduce the video, press the Escape (Esc) button on your keyboard.) In glaucoma, for still unknown reasons, the fluid drains too slowly out of the eye. As the fluid builds up, the pressure inside the eye rises. Unless this pressure is controlled, it may cause damage to the optic nerve and other parts of the eye and resu

In [23]:
example_indices = [9]
dash_line = '-'.join('' for x in range(100))


for i in range(0,1):
    dialogue = dataset2['train'][0]

    prompt = f"""
Dialogue:

{dialogue}

how to avoid the risk?
"""

    inputs = tokenizer(prompt, return_tensors='pt')
    output = tokenizer.decode(
        model.generate(
            inputs["input_ids"],
            max_new_tokens=50,
        )[0],
        skip_special_tokens=True
    )

    print(dash_line)
    print('Example ', i + 1)
    print(dash_line)
    print(f'INPUT PROMPT:\n{prompt}')
    print(dash_line)
    # print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')
    print(dash_line)
    print(f'MODEL GENERATION - ZERO SHOT:\n{output}\n')

---------------------------------------------------------------------------------------------------
Example  1
---------------------------------------------------------------------------------------------------
INPUT PROMPT:

Dialogue:

{'input': 'What is (are) Glaucoma ?', 'output': "Glaucoma is a group of diseases that can damage the eye's optic nerve and result in vision loss and blindness. While glaucoma can strike anyone, the risk is much greater for people over 60. How Glaucoma Develops  There are several different types of glaucoma. Most of these involve the drainage system within the eye. At the front of the eye there is a small space called the anterior chamber. A clear fluid flows through this chamber and bathes and nourishes the nearby tissues. (Watch the video to learn more about glaucoma. To enlarge the video, click the brackets in the lower right-hand corner. To reduce the video, press the Escape (Esc) button on your keyboard.) In glaucoma, for still unknown reasons, the 

<a name='8'></a>
### 8 - Self Conversation Chat Information Retrieval

In [24]:
class ChatApp:
    def __init__(self):
        self.conversation = []

        self.text_area = widgets.Textarea(
            value='',
            placeholder='Conversation will appear here...',
            description='',
            disabled=True,
            layout=widgets.Layout(width='80%', height='300px')
        )
        self.entry = widgets.Text(
            value='',
            placeholder='Type your message here...',
            description='Message:',
            layout=widgets.Layout(width='80%')
        )
        self.send_button = widgets.Button(
            description='Send',
            button_style='info',
            layout=widgets.Layout(width='10%')
        )
        self.save_button = widgets.Button(
            description='Save Conversation',
            button_style='success',
            layout=widgets.Layout(width='20%')
        )

        self.send_button.on_click(self.send_message)
        self.save_button.on_click(self.save_conversation)

        display(self.text_area)
        display(widgets.HBox([self.entry, self.send_button]))
        display(self.save_button)

    def send_message(self, b):
        message = self.entry.value.strip()
        if message:
            if len(self.conversation) % 2 == 0:
                sender = "User 1"
            else:
                sender = "User 2"

            self.conversation.append({"sender": sender, "message": message})
            self.update_chat_area(sender, message)
            self.entry.value = ''

    def update_chat_area(self, sender, message):
        self.text_area.value += f"{sender}: {message}\n"

    def save_conversation(self, b):
        with open("conversation.json", "w") as f:
            json.dump(self.conversation, f, indent=4)
        print("Conversation saved successfully!")

app = ChatApp()

Textarea(value='', disabled=True, layout=Layout(height='300px', width='80%'), placeholder='Conversation will a…

Button(button_style='success', description='Save Conversation', layout=Layout(width='20%'), style=ButtonStyle(…

Conversation saved successfully!


In [25]:
import json

# Function to read and print a JSON file
def print_json_file(file_path):
    try:
        # Open the JSON file
        with open(file_path, 'r') as f:
            # Load the contents of the file
            data = json.load(f)

        # Print the contents in a human-readable format
        print(json.dumps(data, indent=4))
    except FileNotFoundError:
        print(f"The file {file_path} does not exist.")
    except json.JSONDecodeError:
        print(f"Error decoding JSON from the file {file_path}.")

# Specify the path to the JSON file
json_file_path = '/content/conversation.json'

# Call the function to print the JSON file
print_json_file(json_file_path)


[
    {
        "sender": "User 1",
        "message": "How are you Faheem?"
    },
    {
        "sender": "User 2",
        "message": "I am a fine friend. What about you?"
    },
    {
        "sender": "User 1",
        "message": "I am fine too. What do you plan to do for the coming weekend?"
    },
    {
        "sender": "User 2",
        "message": "Nothing special. Perhaps, I will write a poem for the magazine."
    },
    {
        "sender": "User 1",
        "message": "Oh, so you are still writing poems for that magazine. Nice. But I have another plan."
    },
    {
        "sender": "User 2",
        "message": "What's that?"
    },
    {
        "sender": "User 1",
        "message": "I and Junaid have decided to go for a picnic this weekend. Actually, we have also invited Ahmad and he will also join."
    },
    {
        "sender": "User 2",
        "message": "Wow, where you plan to go? I mean some park etc?"
    },
    {
        "sender": "User 1",
        "message": "

In [26]:
# Manualling saving for ease of understanding for now
data_Conversation =  [
    {
        "sender": "User 1",
        "message": "How are you Faheem?"
    },
    {
        "sender": "User 2",
        "message": "I am a fine friend. What about you?"
    },
    {
        "sender": "User 1",
        "message": "I am fine too. What do you plan to do for the coming weekend?"
    },
    {
        "sender": "User 2",
        "message": "Nothing special. Perhaps, I will write a poem for the magazine."
    },
    {
        "sender": "User 1",
        "message": "Oh, so you are still writing poems for that magazine. Nice. But I have another plan."
    },
    {
        "sender": "User 2",
        "message": "What's that?"
    },
    {
        "sender": "User 1",
        "message": "I and Junaid have decided to go for a picnic this weekend. Actually, we have also invited Ahmad and he will also join."
    },
    {
        "sender": "User 2",
        "message": "Wow, where you plan to go? I mean some park etc?"
    },
    {
        "sender": "User 1",
        "message": "No, we have decided to go Indo-Pak border on the edge of river Satluj near Kamran's village."
    },
    {
        "sender": "User 2",
        "message": "Wow, good. May I join you people?"
    },
    {
        "sender": "User 1",
        "message": "Sure. In fact, I have come here to invite you to this trip. We all will go by Junaid's car. After reaching his village Kamran will also join us."
    }
]

In [31]:
example_indices = [9]
dash_line = '-'.join('' for x in range(100))


for i in range(0,1):
    dialogue = data_Conversation

    prompt = f"""
Dialogue:

{dialogue}

Where they are planning the trip ?
"""

    inputs = tokenizer(prompt, return_tensors='pt')
    output = tokenizer.decode(
        model.generate(
            inputs["input_ids"],
            max_new_tokens=50,
        )[0],
        skip_special_tokens=True
    )

    print(dash_line)
    print('Example ', i + 1)
    print(dash_line)
    print(f'INPUT PROMPT:\n{prompt}')
    print(dash_line)
    # print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')
    print(dash_line)
    print(f'MODEL GENERATION:\n{output}\n')

---------------------------------------------------------------------------------------------------
Example  1
---------------------------------------------------------------------------------------------------
INPUT PROMPT:

Dialogue:

[{'sender': 'User 1', 'message': 'How are you Faheem?'}, {'sender': 'User 2', 'message': 'I am a fine friend. What about you?'}, {'sender': 'User 1', 'message': 'I am fine too. What do you plan to do for the coming weekend?'}, {'sender': 'User 2', 'message': 'Nothing special. Perhaps, I will write a poem for the magazine.'}, {'sender': 'User 1', 'message': 'Oh, so you are still writing poems for that magazine. Nice. But I have another plan.'}, {'sender': 'User 2', 'message': "What's that?"}, {'sender': 'User 1', 'message': 'I and Junaid have decided to go for a picnic this weekend. Actually, we have also invited Ahmad and he will also join.'}, {'sender': 'User 2', 'message': 'Wow, where you plan to go? I mean some park etc?'}, {'sender': 'User 1', 'mess